In [9]:
# 基本的なライブラリ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import random
import os

from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from sklearn.metrics import mean_absolute_percentage_error

    
# モデル関連
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

# モデル評価・前処理関連
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, cross_val_score, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# ハイパーパラメータ最適化関連
import optuna
from optuna.integration import LightGBMPruningCallback, XGBoostPruningCallback
import optuna.visualization as vis

# その他のユーティリティ
import shap
import joblib

# 乱数のシードを固定
def torch_fix_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = 'true'
    os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
    
    # Python random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms = True
    torch.backends.cudnn.benchmark = False
    
torch_fix_seed()

# 警告を無視
warnings.filterwarnings("ignore")

In [10]:
# データの読み込み
all_data = pd.read_csv('../data_processed/all_data.csv')

train_data = all_data[0:27532]
X = train_data.drop(['price'], axis=1)
y = train_data['price']

# データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
torch_fix_seed()
def objective(trial):
    # モデルの選択はTabNetのみとする
    model_type = trial.suggest_categorical('model_type', ['tabnet'])
    
    # 交差検証の設定
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    mape_scores = []
    
    for train_idx, val_idx in kf.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx].values.reshape(-1, 1), y.iloc[val_idx].values.reshape(-1, 1)
        
        if model_type == 'tabnet':
            params = {
                'n_d': trial.suggest_int('n_d', 8, 64),
                'n_a': trial.suggest_int('n_a', 8, 64),
                'n_steps': trial.suggest_int('n_steps', 3, 10),
                'gamma': trial.suggest_float('gamma', 1.0, 2.0),
                'n_independent': trial.suggest_int('n_independent', 1, 5),
                'n_shared': trial.suggest_int('n_shared', 1, 5),
                'lambda_sparse': trial.suggest_float('lambda_sparse', 1e-5, 1e-1, log=True),
                'optimizer_fn': torch.optim.Adam,
                'mask_type': trial.suggest_categorical('mask_type', ['entmax', 'sparsemax']),
                'scheduler_params': {"step_size":10, "gamma":0.9},
                'scheduler_fn': torch.optim.lr_scheduler.StepLR,
                'verbose': 0
            }
            model = TabNetRegressor(**params)
            model.fit(X_train.values, y_train, 
                    eval_set=[(X_val.values, y_val)], 
                    patience=10, 
                    batch_size=256, 
                    virtual_batch_size=128, 
                    max_epochs=1000, 
                    num_workers=0, 
                    drop_last=False)

        
        preds = model.predict(X_val.values)
        
        mape = mean_absolute_percentage_error(y_val, preds)
        mape_scores.append(mape)
    
    # 交差検証のMAPEの平均を返す
    return sum(mape_scores) / n_splits

In [13]:
torch_fix_seed()

# Optunaのスタディを作成
study = optuna.create_study(direction='minimize', 
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), 
                            sampler=optuna.samplers.TPESampler(seed=SEED))

# 初期のトライアルを追加
study.enqueue_trial({'model_type': 'tabnet', 'n_d': 64, 'n_a': 14, 'n_steps': 5, 'gamma': 1.5656809766207842, 'n_independent': 4, 'n_shared': 5, 'lambda_sparse': 0.0025180489399281207, 'mask_type': 'entmax'})

# 最適化を実行  
study.optimize(objective, 
               n_trials=None, 
               timeout=60*60*6, 
               n_jobs=4, 
               gc_after_trial=False, 
               show_progress_bar=True)   # 合計のトライアル数とタイムアウトを調整

print('  Value: ', study.best_trial.value)
print('Best trial:', study.best_trial.params)

[I 2023-08-17 00:30:20,643] A new study created in memory with name: no-name-283d35c5-3176-46af-8396-b80fef48fa8e



Early stopping occurred at epoch 26 with best_epoch = 16 and best_val_0_mse = 71071338.62473

Early stopping occurred at epoch 29 with best_epoch = 19 and best_val_0_mse = 71792301.06186

Early stopping occurred at epoch 26 with best_epoch = 16 and best_val_0_mse = 73511713.87505

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_0_mse = 123274555.66668

Early stopping occurred at epoch 34 with best_epoch = 24 and best_val_0_mse = 74057848.43874

Early stopping occurred at epoch 52 with best_epoch = 42 and best_val_0_mse = 70333445.61593

Early stopping occurred at epoch 27 with best_epoch = 17 and best_val_0_mse = 75124980.64166


[I 2023-08-17 01:16:42,244] Trial 3 finished with value: 0.7370319404937166 and parameters: {'model_type': 'tabnet', 'n_d': 47, 'n_a': 8, 'n_steps': 8, 'gamma': 1.416323080416372, 'n_independent': 4, 'n_shared': 5, 'lambda_sparse': 0.020347258847848496, 'mask_type': 'entmax'}. Best is trial 3 with value: 0.7370319404937166.



Early stopping occurred at epoch 46 with best_epoch = 36 and best_val_0_mse = 73449404.26127


[I 2023-08-17 01:23:37,543] Trial 0 finished with value: 0.6941326287151591 and parameters: {'model_type': 'tabnet', 'n_d': 64, 'n_a': 14, 'n_steps': 5, 'gamma': 1.5656809766207842, 'n_independent': 4, 'n_shared': 5, 'lambda_sparse': 0.0025180489399281207, 'mask_type': 'entmax'}. Best is trial 0 with value: 0.6941326287151591.


...


Early stopping occurred at epoch 28 with best_epoch = 18 and best_val_0_mse = 85236400.42004

Early stopping occurred at epoch 21 with best_epoch = 11 and best_val_0_mse = 75545231.17682


[I 2023-08-17 06:39:03,422] Trial 38 finished with value: 0.8401643165653286 and parameters: {'model_type': 'tabnet', 'n_d': 46, 'n_a': 37, 'n_steps': 4, 'gamma': 1.8065770736230689, 'n_independent': 5, 'n_shared': 3, 'lambda_sparse': 0.00023487837588713047, 'mask_type': 'entmax'}. Best is trial 14 with value: 0.6601100784925537.



Early stopping occurred at epoch 35 with best_epoch = 25 and best_val_0_mse = 73932685.3705


[I 2023-08-17 06:40:58,906] Trial 39 finished with value: 0.7097677051162301 and parameters: {'model_type': 'tabnet', 'n_d': 47, 'n_a': 37, 'n_steps': 4, 'gamma': 1.622831742522124, 'n_independent': 5, 'n_shared': 2, 'lambda_sparse': 0.000411742092449686, 'mask_type': 'entmax'}. Best is trial 14 with value: 0.6601100784925537.


  Value:  0.6601100784925537
Best trial: {'model_type': 'tabnet', 'n_d': 32, 'n_a': 63, 'n_steps': 3, 'gamma': 1.9074343286119104, 'n_independent': 2, 'n_shared': 1, 'lambda_sparse': 0.0001850510594279092, 'mask_type': 'entmax'}


In [ ]:
torch_fix_seed()
# 最適なモデルの取得
best_params = study.best_trial.params

model_type = best_params['model_type']

del best_params['model_type']  # モデルタイプをパラメータから削除

model_name = "tabnet"

if model_type == 'tabnet':
    best_model = TabNetRegressor(**best_params)
    best_model.fit(X.values, y.values.reshape(-1, 1)) 
    # tabnetのモデルを保存
    best_model.save_model(f'../model/{model_name}')


epoch 0  | loss: 299110348.30769|  0:00:02s
epoch 1  | loss: 291769335.38462|  0:00:05s
epoch 2  | loss: 275099828.92308|  0:00:07s
epoch 3  | loss: 250504219.07692|  0:00:10s
epoch 4  | loss: 221660050.46154|  0:00:13s
epoch 5  | loss: 191881468.92308|  0:00:15s
epoch 6  | loss: 158011793.84615|  0:00:18s
epoch 7  | loss: 121822026.15385|  0:00:20s
epoch 8  | loss: 101207364.61538|  0:00:23s
epoch 9  | loss: 90795498.76923|  0:00:26s
epoch 10 | loss: 85143535.07692|  0:00:29s
epoch 11 | loss: 82072202.15385|  0:00:32s
epoch 12 | loss: 81663408.61538|  0:00:34s
epoch 13 | loss: 80488378.76923|  0:00:41s
epoch 14 | loss: 80065553.38462|  0:00:45s
epoch 15 | loss: 79608385.23077|  0:00:48s
epoch 16 | loss: 79324316.15385|  0:00:51s
epoch 17 | loss: 78512828.30769|  0:00:53s
epoch 18 | loss: 80437590.15385|  0:00:57s
epoch 19 | loss: 81509268.61538|  0:01:01s
epoch 20 | loss: 79680208.0|  0:01:04s
epoch 21 | loss: 78216952.92308|  0:01:07s
epoch 22 | loss: 78568088.30769|  0:01:10s
epoch 